In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, cross_val_predict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# DO NOT run this every time!
df_og = pd.read_table('P04.awm', sep='\s+', engine='python')
df_og.to_pickle('P04.pkl')

In [ ]:
df_og = pd.read_pickle('P04.pkl')
df_og    # WMSD after KDE

In [ ]:
df = df_og[['Radius', 'ANT', 'GXYX', 'CURVE', 'WellMicroSeismicData']]
df.describe()

In [ ]:
df.corr()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
axes[0].scatter(df['Radius'], df['WellMicroSeismicData'], alpha=0.01)
axes[1].scatter(df['ANT'], df['WellMicroSeismicData'], alpha=0.01)
axes[2].scatter(df['GXYX'], df['WellMicroSeismicData'], alpha=0.01)
axes[3].scatter(df['CURVE'], df['WellMicroSeismicData'], alpha=0.01)

In [ ]:
# DO NOT run this every time!
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)    # 27264; 6816
train_set.to_pickle('train_set.pkl')
test_set.to_pickle('test_set.pkl')
train_set

In [ ]:
train_set = pd.read_pickle('train_set.pkl')
test_set = pd.read_pickle('test_set.pkl')

X = df[['Radius', 'ANT', 'GXYX', 'CURVE']].copy()
y = df['WellMicroSeismicData'].copy()
X_train = train_set[['Radius', 'ANT', 'GXYX', 'CURVE']].copy()    # X_train: feature vairables in training dataset
y_train = train_set['WellMicroSeismicData'].copy()    # y_train : response variable in training dataset
X_test = test_set[['Radius', 'ANT', 'GXYX', 'CURVE']].copy()    # X_test: feature vairables in testing dataset
y_test = test_set['WellMicroSeismicData'].copy()    # y_test : response variable in testing dataset
X_train

In [ ]:
X_s = StandardScaler().fit_transform(X)
X_train_s = StandardScaler().fit_transform(X_train)
X_test_s = StandardScaler().fit_transform(X_test)
X_s

### Regressions Using SciKit-Learn

Models:
- Linear
- Polynomial
- Decision Tree, Decision Tree with Bagging, Decision Tree with Adaptive Boosting, Random Forest
- Extra Trees, Extra Trees with Bagging, Extra Trees with Adaptive Boosting
- Gradient Boosting

Metrics:
- MSE
- R2
- Observed-Predict Plot


**Final Decision**: 
1. Extra Trees with Bagging
2. Random Forest

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train_s, y_train)

lin_pred = lin_reg.predict(X_train_s)
lin_mse = mean_squared_error(y_train, lin_pred)
lin_r2 = r2_score(y_train, lin_pred)

lin_pred_test = lin_reg.predict(X_test_s)
lin_mse_test = mean_squared_error(y_test, lin_pred_test)
lin_r2_test = r2_score(y_test, lin_pred_test)


print(lin_mse, lin_r2, lin_mse_test, lin_r2_test)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, lin_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, lin_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=4)
X_train_poly = poly_features.fit_transform(X_train_s)
X_test_poly = poly_features.fit_transform(X_test_s)

poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)

poly_pred = poly_reg.predict(X_train_poly)
poly_mse = mean_squared_error(y_train, poly_pred)
poly_r2 = r2_score(y_train, poly_pred)

poly_pred_test = poly_reg.predict(X_test_poly)
poly_mse_test = mean_squared_error(y_test, poly_pred_test)
poly_r2_test = r2_score(y_test, poly_pred_test)


print(poly_mse, poly_r2, poly_mse_test, poly_r2_test)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, poly_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, poly_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train_s, y_train)

tree_pred = tree_reg.predict(X_train_s)
tree_mse = mean_squared_error(y_train, tree_pred)
tree_r2 = r2_score(y_train, tree_pred)

tree_pred_test = tree_reg.predict(X_test_s)
tree_mse_test = mean_squared_error(y_test, tree_pred_test)
tree_r2_test = r2_score(y_test, tree_pred_test)


print(tree_mse, tree_r2, tree_mse_test, tree_r2_test, tree_reg.feature_importances_)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, tree_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, tree_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

treebag_reg = BaggingRegressor(DecisionTreeRegressor(), bootstrap=True)
treebag_reg.fit(X_train_s, y_train)

treebag_pred = treebag_reg.predict(X_train_s)
treebag_mse = mean_squared_error(y_train, treebag_pred)
treebag_r2 = r2_score(y_train, treebag_pred)

treebag_pred_test = treebag_reg.predict(X_test_s)
treebag_mse_test = mean_squared_error(y_test, treebag_pred_test)
treebag_r2_test = r2_score(y_test, treebag_pred_test)


print(treebag_mse, treebag_r2, treebag_mse_test, treebag_r2_test)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, treebag_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, treebag_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

treeada_reg = AdaBoostRegressor(DecisionTreeRegressor())
treeada_reg.fit(X_train_s, y_train)

treeada_pred = treeada_reg.predict(X_train_s)
treeada_mse = mean_squared_error(y_train, treeada_pred)
treeada_r2 = r2_score(y_train, treeada_pred)

treeada_pred_test = treeada_reg.predict(X_test_s)
treeada_mse_test = mean_squared_error(y_test, treeada_pred_test)
treeada_r2_test = r2_score(y_test, treeada_pred_test)


print(treeada_mse, treeada_r2, treeada_mse_test, treeada_r2_test, treeada_reg.feature_importances_)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, treeada_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, treeada_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor()
rf_reg.fit(X_train_s, y_train)

rf_pred = rf_reg.predict(X_train_s)
rf_mse = mean_squared_error(y_train, rf_pred)
rf_r2 = r2_score(y_train, rf_pred)

rf_pred_test = rf_reg.predict(X_test_s)
rf_mse_test = mean_squared_error(y_test, rf_pred_test)
rf_r2_test = r2_score(y_test, rf_pred_test)


print(rf_mse, rf_r2, rf_mse_test, rf_r2_test, rf_reg.feature_importances_)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, rf_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, rf_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

et_reg = ExtraTreesRegressor()
et_reg.fit(X_train_s, y_train)

et_pred = et_reg.predict(X_train_s)
et_mse = mean_squared_error(y_train, et_pred)
et_r2 = r2_score(y_train, et_pred)

et_pred_test = et_reg.predict(X_test_s)
et_mse_test = mean_squared_error(y_test, et_pred_test)
et_r2_test = r2_score(y_test, et_pred_test)


print(et_mse, et_r2, et_mse_test, et_r2_test, et_reg.feature_importances_)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, et_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, et_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor

etbag_reg = BaggingRegressor(ExtraTreesRegressor())
etbag_reg.fit(X_train_s, y_train)

etbag_pred = etbag_reg.predict(X_train_s)
etbag_mse = mean_squared_error(y_train, etbag_pred)
etbag_r2 = r2_score(y_train, etbag_pred)

etbag_pred_test = etbag_reg.predict(X_test_s)
etbag_mse_test = mean_squared_error(y_test, etbag_pred_test)
etbag_r2_test = r2_score(y_test, etbag_pred_test)


print(etbag_mse, etbag_r2, etbag_mse_test, etbag_r2_test)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, etbag_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, etbag_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor

etada_reg = AdaBoostRegressor(ExtraTreesRegressor())
etada_reg.fit(X_train_s, y_train)

etada_pred = etada_reg.predict(X_train_s)
etada_mse = mean_squared_error(y_train, etada_pred)
etada_r2 = r2_score(y_train, etada_pred)

etada_pred_test = etada_reg.predict(X_test_s)
etada_mse_test = mean_squared_error(y_test, etada_pred_test)
etada_r2_test = r2_score(y_test, etada_pred_test)


print(etada_mse, etada_r2, etada_mse_test, etada_r2_test, etada_reg.feature_importances_)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, etada_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, etada_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_reg = GradientBoostingRegressor(warm_start=True, subsample=0.25)
gb_reg.fit(X_train_s, y_train)

gb_pred = gb_reg.predict(X_train_s)
nn_mse = mean_squared_error(y_train, gb_pred)
gb_r2 = r2_score(y_train, gb_pred)

gb_pred_test = gb_reg.predict(X_test_s)
gb_mse_test = mean_squared_error(y_test, gb_pred_test)
gb_r2_test = r2_score(y_test, gb_pred_test)


print(nn_mse, gb_r2, gb_mse_test, gb_r2_test, gb_reg.feature_importances_)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, gb_pred, alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, gb_pred_test, alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))

### PCA using SciKit-Learn

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train_s)
plt.plot(pca.explained_variance_ratio_)
pca.explained_variance_ratio_, pca.components_

X_train_pca = pca.fit_transform(X_train_s)

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(12, 9))

# 1st
axes[0,0].scatter(X_train_pca[:,0], X_train_pca[:,1], c=y_train, alpha=0.05)

axes[0,1].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),0], X_train_pca[(y_train>0.0) & (y_train<0.1),1], alpha=0.05)

axes[0,2].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),0], X_train_pca[(y_train>0.0) & (y_train<0.1),1], alpha=0.05)
axes[0,2].scatter(X_train_pca[(y_train>0.1) & (y_train<0.2),0], X_train_pca[(y_train>0.1) & (y_train<0.2),1], alpha=0.05)

axes[0,3].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),0], X_train_pca[(y_train>0.0) & (y_train<0.1),1], alpha=0.05)
axes[0,3].scatter(X_train_pca[(y_train>0.1) & (y_train<0.2),0], X_train_pca[(y_train>0.1) & (y_train<0.2),1], alpha=0.05)
axes[0,3].scatter(X_train_pca[(y_train>0.2) & (y_train<0.4),0], X_train_pca[(y_train>0.2) & (y_train<0.4),1], alpha=0.05)

# 2nd
axes[1,0].scatter(X_train_pca[:,0], X_train_pca[:,2], c=y_train, alpha=0.05)

axes[1,1].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),0], X_train_pca[(y_train>0.0) & (y_train<0.1),2], alpha=0.05)

axes[1,2].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),0], X_train_pca[(y_train>0.0) & (y_train<0.1),2], alpha=0.05)
axes[1,2].scatter(X_train_pca[(y_train>0.1) & (y_train<0.2),0], X_train_pca[(y_train>0.1) & (y_train<0.2),2], alpha=0.05)

axes[1,3].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),0], X_train_pca[(y_train>0.0) & (y_train<0.1),2], alpha=0.05)
axes[1,3].scatter(X_train_pca[(y_train>0.1) & (y_train<0.2),0], X_train_pca[(y_train>0.1) & (y_train<0.2),2], alpha=0.05)
axes[1,3].scatter(X_train_pca[(y_train>0.2) & (y_train<0.4),0], X_train_pca[(y_train>0.2) & (y_train<0.4),2], alpha=0.05)

#3rd
axes[2,0].scatter(X_train_pca[:,1], X_train_pca[:,2], c=y_train, alpha=0.05)

axes[2,1].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),1], X_train_pca[(y_train>0.0) & (y_train<0.1),2], alpha=0.05)

axes[2,2].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),1], X_train_pca[(y_train>0.0) & (y_train<0.1),2], alpha=0.05)
axes[2,2].scatter(X_train_pca[(y_train>0.1) & (y_train<0.2),1], X_train_pca[(y_train>0.1) & (y_train<0.2),2], alpha=0.05)

axes[2,3].scatter(X_train_pca[(y_train>0.0) & (y_train<0.1),1], X_train_pca[(y_train>0.0) & (y_train<0.1),2], alpha=0.05)
axes[2,3].scatter(X_train_pca[(y_train>0.1) & (y_train<0.2),1], X_train_pca[(y_train>0.1) & (y_train<0.2),2], alpha=0.05)
axes[2,3].scatter(X_train_pca[(y_train>0.2) & (y_train<0.4),1], X_train_pca[(y_train>0.2) & (y_train<0.4),2], alpha=0.05)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection = '3d')
ax.scatter(X_train_pca[(y_train>0.0) & (y_train<0.1), 0], X_train_pca[(y_train>0.0) & (y_train<0.1), 1], X_train_pca[(y_train>0.0) & (y_train<0.1), 2], alpha = 0.01)
ax.scatter(X_train_pca[(y_train>0.1) & (y_train<0.2), 0], X_train_pca[(y_train>0.1) & (y_train<0.2), 1], X_train_pca[(y_train>0.1) & (y_train<0.2), 2], alpha = 0.01)
ax.scatter(X_train_pca[(y_train>0.2) & (y_train<0.4), 0], X_train_pca[(y_train>0.2) & (y_train<0.4), 1], X_train_pca[(y_train>0.2) & (y_train<0.4), 2], alpha = 0.01)

### Neural Network Regression Using PyTorch

In [ ]:
X_train_t = torch.tensor(X_train_s, dtype=torch.float32)
y_train_t = torch.tensor(np.array(y_train), dtype=torch.float32)
train_t = TensorDataset(X_train_t, y_train_t)
trainloader = DataLoader(train_t, shuffle=True)

X_test_t = torch.tensor(X_test_s, dtype=torch.float32)
y_test_t = torch.tensor(np.array(y_test), dtype=torch.float32)
test_t = TensorDataset(X_test_t, y_test_t)
testloader = DataLoader(test_t, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        
    def forward(self, x):
        f1 = F.relu(self.fc1(x))
        f2 = F.relu(self.fc2(f1))
        f3 = F.relu(self.fc3(f2))
        output = self.fc4(f3).reshape(-1)
        return output
    
net = Net()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)


In [ ]:
net.train()
for epoch in range(40):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10000 == 9999:    # print every ...
            print(f'[{epoch + 1},{i + 1:5d}] loss: {running_loss / 2000:.6f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
net.eval()
with torch.no_grad():
    nn_pred = net(X_train_t)
    nn_pred_test = net(X_test_t)
    test_loss = criterion(nn_pred_test, y_test_t)

nn_mse = mean_squared_error(y_train, nn_pred)
nn_r2 = r2_score(y_train, nn_pred)

nn_mse_test = mean_squared_error(y_test, nn_pred_test)
nn_r2_test = r2_score(y_test, nn_pred_test)

print(nn_mse, nn_mse_test, nn_r2, nn_r2_test)
print(f'Test Loss: {test_loss.item(): .4f}')

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(y_train, nn_pred.detach().numpy(), alpha = 0.02)
axes[0].plot((0, 0.4), (0, 0.4))
axes[1].scatter(y_test, nn_pred_test.detach().numpy(), alpha = 0.05)
axes[1].plot((0, 0.4), (0, 0.4))


# 100e: 0.011220 0.0022 0.6686302542089703 0.6467765149613168


In [ ]:
torch.save(net.state_dict(), '100e.pth')